In [75]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.io import fits

Using matplotlib backend: Qt5Agg


In [76]:
# Reading in Machine Learning and Spectroscopy Data
spectroscopy_data = pd.read_csv('/home/shawn/Downloads/final_sample.csv')
machine_learning_data = pd.read_csv('/home/shawn/Downloads/linkstable_trimmed.csv')
spec = spectroscopy_data
mac = machine_learning_data

# Spec : 122 total objects
# Mac : 1983 total objects

#print(spec.columns, mac.columns)
#print(len(spec),len(mac))

In [77]:
# Opening Galaxy Zoo Data... data is the table. zoo_survey_id is the 
# GAMA ID listed in the galaxy zoo catalog.
hdul = fits.open('/home/shawn/Downloads/galaxy_zoo_gama_cleaned.fits')  # open a FITS file
zoo_data = hdul[1].data  # assume the first extension is a table

zoo_survey_id = zoo_data['survey_id'] # GAMA IDs
zoo_oddtype_lens_arc_frac = zoo_data['oddtype_lens_or_arc_frac'] #score

hdul.close()

#print(zoo_data.columns)

#print(len(zoo_oddtype_lens_arc_frac))

# Histogram of Lens Scores
plt.xlabel("Fraction of Participants who Identified Lens Features")
plt.ylabel("Number of Candidate Samples")
plt.title("Galaxy Zoo Lens Selection Scores")
plt.hist(zoo_oddtype_lens_arc_frac,range=[0.05,0.7],bins=200,color='wheat')

(array([359., 396., 402., 412., 386., 418., 429., 424., 397., 393., 337.,
        368., 361., 297., 287., 256., 209., 254., 229., 173., 200., 140.,
        209., 129., 124., 109., 120., 120., 124.,  95.,  80.,  78.,  79.,
         68.,  65.,  79.,  45.,  36.,  56.,  39.,  47.,  39.,  33.,  34.,
         38.,  61.,  48.,  27.,  22.,  24.,  20.,  14.,  19.,  14.,  14.,
         17.,  18.,  10.,  15.,  15.,  24.,  19.,  11.,  11.,   9.,   8.,
          7.,   3.,   6.,   6.,   6.,   7.,   5.,   4.,   3.,   6.,   6.,
          6.,   3.,   3.,   2.,   8.,   4.,   5.,   1.,   4.,   8.,   7.,
          2.,   0.,   1.,   2.,   2.,   2.,   2.,   1.,   1.,   2.,   1.,
          0.,   1.,   2.,   1.,   1.,   2.,   1.,   0.,   1.,   0.,   2.,
          0.,   1.,   0.,   3.,   2.,   1.,   1.,   0.,   0.,   0.,   0.,
          0.,   0.,   1.,   1.,   2.,   1.,   1.,   0.,   0.,   1.,   2.,
          0.,   0.,   2.,   0.,   0.,   1.,   1.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   1.,   1., 

In [78]:
# Opening GAMA data from G09, G12, and G15 (DR3)
hdul = fits.open('/home/shawn/Downloads/SpecAll.fits')  # open a FITS file
hdul.verify('fix')
gama = hdul[1].data  # assume the first extension is a table
#print(hdul[1].columns)

gama_id = gama['CATAID']
gama_ra = gama['RA']
gama_dec = gama['DEC']

#print(len(gama_id))

In [79]:
# Cross-correlating the gama_id from the GAMA data with the zoo_survey_id
# from the Galaxy Zoo data.
zoo_survey_id_integer = [int(zoo_survey_id) for zoo_survey_id in zoo_survey_id]

# Matches GAMA ID from GAMA with Survey ID in Zoo
where_galaxyzoo = np.in1d(gama_id,zoo_survey_id_integer)

# Creates three new list variables with the matched values.
galaxyzoo_gama_id = gama_id[where_galaxyzoo]
galaxyzoo_ra = gama_ra[where_galaxyzoo]
galaxyzoo_dec = gama_dec[where_galaxyzoo]

# Zips the three lists and then creates a dataframe with those as columns
zoo_gamaid_ra_dec_data = list(zip(galaxyzoo_gama_id, galaxyzoo_ra, galaxyzoo_dec))
zoo_gamaid_ra_dec = pd.DataFrame(zoo_gamaid_ra_dec_data, columns =["GAMA_ID","RA","DEC"])
#print(zoo_gamaid_ra_dec)

# Zips two lists and creates a dataframe with those as columns.
zoo_surveyid_lensfrac_data = list(zip(zoo_survey_id_integer, zoo_oddtype_lens_arc_frac))
zoo_surveyid_lensfrac = pd.DataFrame(zoo_surveyid_lensfrac_data, columns = ["GAMA_ID", "LENS_SCORE"])
#(zoo_surveyid_lensfrac)

# Merges dataframes to create Zoo in Gama.
zoo_in_gama = pd.merge(zoo_gamaid_ra_dec, zoo_surveyid_lensfrac, on ="GAMA_ID")

In [80]:
# Cleans up Zoo data.
zoo_in_gama.drop_duplicates('GAMA_ID', inplace = True)

In [81]:
# Cleans up Zoo data.
zoo_in_gama.reset_index(inplace=True)

In [82]:
# Cleans up Zoo data.
zoo_in_gama.drop(columns=['index'], inplace = True)

In [83]:
### 38769 total objects from Galaxy Zoo.
zoo_in_gama

,GAMA_ID,RA,DEC,LENS_SCORE
0,491457,211.73487,-1.59471,0.000000
1,460298,211.51452,-1.65754,0.000000
2,460345,211.78462,-1.71899,0.000000
3,484623,211.63878,-1.93563,0.000000
4,484637,211.79006,-1.89456,0.169778
5,491454,211.71473,-1.54075,0.000000
6,491414,211.50587,-1.53749,0.000000
7,484621,211.63321,-1.96885,0.061127
8,463119,211.54913,-1.22789,0.000000
9,463148,211.68703,-1.33978,0.143398


In [84]:
# Identifying where Zoo Score > 0
zoo_in_gama_found_lens = zoo_in_gama[zoo_in_gama.LENS_SCORE != 0]

# Identifying where Zoo Score > 20
zoo_in_gama_above_twenty = zoo_in_gama[zoo_in_gama.LENS_SCORE >= 0.2]

In [85]:
# Cleans up Zoo sublists. Run only once.
zoo_in_gama_found_lens.reset_index(inplace = True)

In [86]:
# Cleans up Zoo sublists. Run only once.
zoo_in_gama_found_lens.drop(columns=['index'],inplace = True)

In [87]:
# Cleans up Zoo sublists. Run only once.
zoo_in_gama_above_twenty.reset_index(inplace = True)

In [88]:
# Cleans up Zoo sublists. Run only once.
zoo_in_gama_above_twenty.drop(columns=['index'],inplace = True)

In [89]:
# Zoo identified 12933 lenses with score above 0.
zoo_in_gama_found_lens

,GAMA_ID,RA,DEC,LENS_SCORE
0,484637,211.79006,-1.89456,0.169778
1,484621,211.63321,-1.96885,0.061127
2,463148,211.68703,-1.33978,0.143398
3,543438,211.53734,-1.03689,0.213034
4,463145,211.63664,-1.23923,0.000755
5,460374,212.03548,-1.70230,0.024226
6,484655,211.88866,-1.97102,0.003699
7,460379,211.94664,-1.71637,0.110855
8,463176,211.81052,-1.19533,0.045798
9,463197,212.00002,-1.29196,0.093139


In [90]:
# Zoo identified 462 lenses with score above 20.
zoo_20 = zoo_in_gama_above_twenty
zoo_20

,GAMA_ID,RA,DEC,LENS_SCORE
0,543438,211.53734,-1.03689,0.213034
1,491933,213.94101,-1.42622,0.236187
2,463601,213.79297,-1.24519,0.221626
3,492121,214.80569,-1.40320,0.237688
4,511867,216.38846,-1.11396,0.352229
5,511865,216.31907,-1.13127,0.315974
6,485529,216.24766,-1.86855,0.246178
7,485866,217.55467,-1.72059,0.202707
8,485873,217.75015,-1.80042,0.349456
9,508674,217.85610,-1.47254,0.214371


In [91]:
# Identifying the lenses found in GAMA Regions G09, G12, and G15 from
# Spec and Mac

spec_in_G09 = spec[(spec.ra >= 129.0) & (spec.ra <= 141.0) & (spec.dec >= -2) & (spec.dec <= 3)]
spec_in_G12 = spec[(spec.ra >= 174.0) & (spec.ra <= 186.0) & (spec.dec >= -3) & (spec.dec <= 2)]                   
spec_in_G15 = spec[(spec.ra >= 211.5) & (spec.ra <= 223.5) & (spec.dec >= -2) & (spec.dec <= 3)]

mac_in_G09 = mac[(mac.RA >= 129.0) & (mac.RA <= 141.0) & (mac.DEC >= -2) & (mac.DEC <= 3)]
mac_in_G12 = mac[(mac.RA >= 174.0) & (mac.RA <= 186.0) & (mac.DEC >= -3) & (mac.DEC <= 2)]
mac_in_G15 = mac[(mac.RA >= 211.5) & (mac.RA <= 223.5) & (mac.DEC >= -2) & (mac.DEC <= 3)]

zoo_20_in_G09 = zoo_20[(zoo_20.RA >= 129.0) & (zoo_20.RA <= 141.0) & (zoo_20.DEC >= -2) & (zoo_20.DEC <= 3)]
zoo_20_in_G12 = zoo_20[(zoo_20.RA >= 174.0) & (zoo_20.RA <= 186.0) & (zoo_20.DEC >= -3) & (zoo_20.DEC <= 2)]
zoo_20_in_G15 = zoo_20[(zoo_20.RA >= 211.5) & (zoo_20.RA <= 223.5) & (zoo_20.DEC >= -2) & (zoo_20.DEC <= 3)]

# Spec cropped to G09, G12, and G15.
spec_data_in_gama = np.vstack((spec_in_G09, spec_in_G12, spec_in_G15))
spec_in_gama = pd.DataFrame(spec_data_in_gama, columns = ["Unnamed: 0","RA","DEC","z1","z2","spec_type","GAMA_ID"])

# Mac cropped to G09, G12, and G15
mac_data_in_gama = np.vstack((mac_in_G09, mac_in_G12, mac_in_G15))
mac_in_gama = pd.DataFrame(mac_data_in_gama, columns = ['ID', 'score', 'P1', 'P3', 'RA', 'DEC', 'DISC'])

In [92]:
# Spec shows 85 objects in G09, G12, and G15
spec_in_gama

,Unnamed: 0,RA,DEC,z1,z2,spec_type,GAMA_ID
0,0,138.281,-0.66673,0.163,0.321,PG+ELG,198082
1,1,129.695,-0.38179,0.418,0.738,PG+ELG,202448
2,2,136.639,-0.35203,0.282,0.449,PG+ELG,204140
3,3,132.368,0.1636,0.128,0.603,PG+ELG,209222
4,4,132.61,0.11972,0.313,0.608,PG+ELG,209295
5,5,134.03,0.15244,0.167,0.158,PG+ELG,209584
6,6,140.142,0.97341,0.586,0.336,PG+ELG,279956
7,7,135.329,1.22984,0.487,0.247,PG+ELG,301818
8,8,138.941,1.33144,0.593,0.404,PG+ELG,302719
9,9,130.737,1.55957,0.416,0.35,PG+ELG,323200


In [93]:
# Mac shows 421 objects in G09, G12, and G15
mac_in_gama

,ID,score,P1,P3,RA,DEC,DISC
0,2251.0,64.0,0.887,0.901,131.33397,-0.91563,0.0
1,2660.0,30.0,0.990,0.989,132.98734,1.50468,0.0
2,2967.0,30.0,1.000,1.000,130.54643,1.64325,0.0
3,2669.0,28.0,NaN,0.858,140.35284,-0.97414,0.0
4,1933.0,26.0,NaN,0.883,135.88864,-0.97487,0.0
5,1906.0,26.0,0.929,NaN,129.23265,-1.57512,0.0
6,2007.0,24.0,0.936,NaN,137.85838,-0.75280,0.0
7,2756.0,22.0,NaN,0.924,134.57492,-1.55142,0.0
8,2366.0,22.0,0.837,NaN,140.47475,-0.10551,0.0
9,1704.0,20.0,0.985,NaN,138.66089,-1.11639,0.0


In [94]:
# Mac and Spec... Columns: GAMA_ID, ID, RA, DEC, MAC_SCORE
mac_and_spec_matches = pd.DataFrame(columns = ['GAMA_ID', 'ID', 'RA', 'DEC', 'MAC_SCORE'], index = range(len(mac_in_gama)))
for i in range(len(mac_in_gama)):
    Rlim = 2./3600.
    for j in range(len(spec_in_gama)):
        R = np.sqrt(np.square(mac_in_gama.DEC[i]-spec_in_gama.DEC[j])+np.square(mac_in_gama.RA[i]-spec_in_gama.RA[j]))
        if R <= Rlim:
            Rlim = R
            mac_and_spec_matches.GAMA_ID[i] = spec_in_gama.GAMA_ID[j]
            mac_and_spec_matches.ID[i] = mac_in_gama.ID[i]
            mac_and_spec_matches.RA[i] = spec_in_gama.RA[j]
            mac_and_spec_matches.DEC[i] = spec_in_gama.DEC[j]
            mac_and_spec_matches.MAC_SCORE[i] = mac_in_gama.score[i]
            print('Ready to roll out!')

mac_and_spec_matches.dropna(inplace = True)

Ready to roll out!
Ready to roll out!
Ready to roll out!
Ready to roll out!


In [126]:
# Found 4 matches between Mac and Spec
#mac_and_spec_matches.to_csv('/home/shawn/Documents/Lens_Project_Files/mac_and_spec_matches.csv')
mac_and_spec_matches

,GAMA_ID,ID,RA,DEC,MAC_SCORE
78,202448,1129,129.695,-0.38179,4
79,323200,3430,130.737,1.55957,4
278,220682,1618,182.744,1.59787,4
368,342308,501,215.069,2.22422,4


In [96]:
# Mac and Zoo... Columns: GAMA ID, ID, RA, DEC, SCORE
mac_and_zoo_matches = pd.DataFrame(columns = ['GAMA_ID', 'ID', 'RA', 'DEC', 'MAC_SCORE', 'ZOO_SCORE'], index = range(len(mac_in_gama)))
for i in range(len(mac_in_gama)):
    Rlim = 2./3600.
    for j in range(len(zoo_in_gama_found_lens)):
        R = np.sqrt(np.square(mac_in_gama.DEC[i]-zoo_in_gama_found_lens.DEC[j])+np.square(mac_in_gama.RA[i]-zoo_in_gama_found_lens.RA[j]))
        if R <= Rlim:
            Rlim = R
            mac_and_zoo_matches.GAMA_ID[i] = zoo_in_gama_found_lens.GAMA_ID[j]
            mac_and_zoo_matches.ID[i] = mac_in_gama.ID[i]
            mac_and_zoo_matches.RA[i] = zoo_in_gama_found_lens.RA[j]
            mac_and_zoo_matches.DEC[i] = zoo_in_gama_found_lens.DEC[j]
            mac_and_zoo_matches.MAC_SCORE[i] = mac_in_gama.score[i]
            mac_and_zoo_matches.ZOO_SCORE[i] = zoo_in_gama_found_lens.LENS_SCORE[j]
            print('Wanna piece of me, boy?')
mac_and_zoo_matches.dropna(inplace = True)

Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?
Wanna piece of me, boy?


In [127]:
# Found 11 matches between Mac and Zoo (1 duplicate)
#mac_and_zoo_matches.to_csv('/home/shawn/Documents/Lens_Project_Files/mac_and_zoo_matches.csv')
mac_and_zoo_matches

,GAMA_ID,ID,RA,DEC,MAC_SCORE,ZOO_SCORE
124,517068,3063,130.594,2.46237,4,0.137209
151,165409,3098,181.197,-2.72003,38,0.0595426
158,124486,2553,179.73,-2.51919,28,0.42619
166,165871,2803,183.702,-2.78805,18,0.282701
168,164145,2805,176.553,-2.77562,16,0.0922649
170,271500,2496,174.511,1.34984,16,0.139357
293,271500,375,174.511,1.34984,4,0.139357
300,136604,1642,175.873,-1.74167,58,0.316451
389,618600,613,216.263,0.26191,4,0.0114096
393,106569,1524,216.665,0.84988,4,0.178933


In [100]:
# Spec and Zoo... Columns : GAMA_ID, RA, DEC, ZOO_SCORE
spec_and_zoo_matches = pd.DataFrame(columns = ['GAMA_ID', 'RA', 'DEC', 'ZOO_SCORE'], index = range(len(spec_in_gama)))
for i in range(len(spec_in_gama)):
    Rlim = 2./3600.
    for j in range(len(zoo_in_gama_found_lens)):
        R = np.sqrt(np.square(spec_in_gama.DEC[i]-zoo_in_gama_found_lens.DEC[j])+np.square(spec_in_gama.RA[i]-zoo_in_gama_found_lens.RA[j]))
        if R <= Rlim:
            Rlim = R
            spec_and_zoo_matches.GAMA_ID[i] = zoo_in_gama_found_lens.GAMA_ID[j]
            spec_and_zoo_matches.RA[i] = zoo_in_gama_found_lens.RA[j]
            spec_and_zoo_matches.DEC[i] = zoo_in_gama_found_lens.DEC[j]
            spec_and_zoo_matches.ZOO_SCORE[i] = zoo_in_gama_found_lens.LENS_SCORE[j]
            print('Wraith awaiting launch orders!')
spec_and_zoo_matches.dropna(inplace = True)

Wraith awaiting launch orders!


In [128]:
# Found 1 match between Spec and Zoo
#spec_and_zoo_matches.to_csv('/home/shawn/Documents/Lens_Project_Files/spec_and_zoo_matches.csv')
spec_and_zoo_matches

,GAMA_ID,RA,DEC,ZOO_SCORE
79,460463,212.481,-1.61746,0.139206


In [102]:
# Plot of three methods by location in three fields. Zoo data 
# is given for all objects that received score > 0
plt.plot(zoo_in_gama_found_lens.RA,zoo_in_gama_found_lens.DEC,'yo',label='Galaxy Zoo')
plt.plot(mac_in_gama.RA,mac_in_gama.DEC,'go',label='Machine Learning')
plt.plot(spec_in_gama.RA,spec_in_gama.DEC,'bo',label='Spectroscopy')

plt.xlabel("Right Ascension")
plt.ylabel("Declination")
plt.title("Three Methods by Location")
plt.legend(loc='best')

plt.show()

In [105]:
# Plots of individual GAMA regions G09, G12, and G15 to show the distribution.
# Plot of G09.
plt.plot(zoo_20_in_G09.RA,zoo_20_in_G09.DEC,'yo',label='Galaxy Zoo')
plt.plot(mac_in_G09.RA,mac_in_G09.DEC,'go',label='Machine Learning')
plt.plot(spec_in_G09.ra,spec_in_G09.dec,'bo',label='GAMA Spectroscopy')

plt.xlabel("Right Ascension")
plt.ylabel("Declination")
plt.title("GAMA Region G09")
plt.legend(loc='best')

plt.show()

# Plot of G12.
plt.plot(zoo_20_in_G12.RA,zoo_20_in_G12.DEC,'yo',label='Galaxy Zoo')
plt.plot(mac_in_G12.RA,mac_in_G12.DEC,'go',label='Machine Learning')
plt.plot(spec_in_G12.ra,spec_in_G12.dec,'bo',label='GAMA Spectroscopy')

plt.xlabel("Right Ascension")
plt.ylabel("Declination")
plt.title("GAMA Region G12")
plt.legend(loc='best')

plt.show()

# Plot of G15.
plt.plot(zoo_20_in_G15.RA,zoo_20_in_G15.DEC,'yo',label='Galaxy Zoo')
plt.plot(mac_in_G15.RA,mac_in_G15.DEC,'go',label='Machine Learning')
plt.plot(spec_in_G15.ra,spec_in_G15.dec,'bo',label='GAMA Spectroscopy')

plt.xlabel("Right Ascension")
plt.ylabel("Declination")
plt.title("GAMA Region G15")
plt.legend(loc='best')

plt.show()

In [106]:
# Opening MagPhys.fits
hdul = fits.open('/home/shawn/Downloads/MagPhys.fits')  # open a FITS file
hdul.verify('fix')
MagPhys = hdul[1].data  # assume the first extension is a table
#print(hdul[1].columns)

In [113]:
# Adding GAMA ID to Mac by comparing to RA/Dec in GAMA data.
mac_in_gama_withgamaid = pd.DataFrame(columns = ['GAMA_ID','ID','RA','DEC','SCORE'], index = range(len(mac_in_gama)))

for i in range(len(mac_in_gama)):
    Rlim = 2./3600.
    for j in range(len(gama)):
        R = np.sqrt(np.square(mac_in_gama.DEC[i]-gama.DEC[j])+np.square(mac_in_gama.RA[i]-gama.RA[j]))
        if R <= Rlim:
            Rlim = R
            mac_in_gama_withgamaid.GAMA_ID[i] = gama.CATAID[j]
            mac_in_gama_withgamaid.ID[i] = mac_in_gama.ID[i]
            mac_in_gama_withgamaid.RA[i] = mac_in_gama.RA[i]
            mac_in_gama_withgamaid.DEC[i] = mac_in_gama.DEC[i]
            mac_in_gama_withgamaid.SCORE[i] = mac_in_gama.score[i]
            print('My life for Aiur!', i+1, mac_in_gama_withgamaid.GAMA_ID[i])
#mac_in_gama_withgamaid.dropna(inplace = True)

My life for Aiur! 1 3899011
My life for Aiur! 2 376338
My life for Aiur! 3 323152
My life for Aiur! 4 663364
My life for Aiur! 5 3629152
My life for Aiur! 5 3629152
My life for Aiur! 6 3575500
My life for Aiur! 7 197972
My life for Aiur! 8 3883690
My life for Aiur! 10 3911769
My life for Aiur! 10 3911769
My life for Aiur! 10 3911769
My life for Aiur! 11 371400
My life for Aiur! 11 371400
My life for Aiur! 11 371400
My life for Aiur! 12 204703
My life for Aiur! 13 279505
My life for Aiur! 14 386188
My life for Aiur! 14 386188
My life for Aiur! 14 386188
My life for Aiur! 14 386188
My life for Aiur! 15 3599660
My life for Aiur! 16 3891172
My life for Aiur! 16 3891172
My life for Aiur! 17 345824
My life for Aiur! 18 3913677
My life for Aiur! 20 3575548
My life for Aiur! 22 3866174
My life for Aiur! 28 3581425
My life for Aiur! 28 3581425
My life for Aiur! 29 3611074
My life for Aiur! 31 600305
My life for Aiur! 35 419170
My life for Aiur! 39 574628
My life for Aiur! 40 347923
My life for 

My life for Aiur! 318 77991
My life for Aiur! 318 77991
My life for Aiur! 319 363092
My life for Aiur! 320 250715
My life for Aiur! 321 594552
My life for Aiur! 322 618700
My life for Aiur! 323 15705
My life for Aiur! 324 911856
My life for Aiur! 325 249961
My life for Aiur! 325 249961
My life for Aiur! 326 278118
My life for Aiur! 327 569022
My life for Aiur! 328 569805
My life for Aiur! 329 261497
My life for Aiur! 330 262874
My life for Aiur! 330 262874
My life for Aiur! 331 250434
My life for Aiur! 332 278049
My life for Aiur! 332 278049
My life for Aiur! 332 278049
My life for Aiur! 333 505067
My life for Aiur! 333 505067
My life for Aiur! 334 237985
My life for Aiur! 335 250585
My life for Aiur! 336 362163
My life for Aiur! 337 79099
My life for Aiur! 338 505502
My life for Aiur! 339 922203
My life for Aiur! 340 511700
My life for Aiur! 341 362429
My life for Aiur! 342 77991
My life for Aiur! 342 77991
My life for Aiur! 343 567883
My life for Aiur! 344 296451
My life for Aiur! 34

In [124]:
#mac_in_gama_withgamaid.to_csv('/home/shawn/Documents/Lens_Project_Files/mac_in_gama_withgamaid.csv')
mac_in_gama_withgamaid

,GAMA_ID,ID,RA,DEC,SCORE
0,3899011,2251,131.334,-0.91563,64
1,376338,2660,132.987,1.50468,30
2,323152,2967,130.546,1.64325,30
3,663364,2669,140.353,-0.97414,28
4,3629152,1933,135.889,-0.97487,26
5,3575500,1906,129.233,-1.57512,26
6,197972,2007,137.858,-0.7528,24
7,3883690,2756,134.575,-1.55142,22
8,NaN,NaN,NaN,NaN,NaN
9,3911769,1704,138.661,-1.11639,20


In [131]:
# Adding MagPhys Mass and Z to Spec, Mac, and Zoo.
magphys_gama_id = MagPhys['CATAID']
stellar_mass = MagPhys['mass_stellar_best_fit']
magphys_redshift = MagPhys['Z']

magphys_data = list(zip(magphys_gama_id, stellar_mass, magphys_redshift))

magphys_id_mass = pd.DataFrame(magphys_data, columns = ['GAMA_ID', 'STELLAR_MASS', 'Z'])

spec_with_mass = pd.merge(spec_in_gama, magphys_id_mass, how = 'left', on = 'GAMA_ID')
zoo_with_mass = pd.merge(zoo_in_gama, magphys_id_mass, how = 'left', on = 'GAMA_ID')
zoo_lens_with_mass = pd.merge(zoo_in_gama_found_lens, magphys_id_mass, how = 'left', on = 'GAMA_ID')
zoo_lens20_with_mass = pd.merge(zoo_20, magphys_id_mass, how = 'left', on = 'GAMA_ID')
mac_with_mass = pd.merge(mac_in_gama_withgamaid, magphys_id_mass, how = 'left', on = 'GAMA_ID')


In [132]:
spec_with_mass

,Unnamed: 0,RA,DEC,z1,z2,spec_type,GAMA_ID,STELLAR_MASS,Z
0,0,138.281,-0.66673,0.163,0.321,PG+ELG,198082,3.025000e+10,0.1632
1,1,129.695,-0.38179,0.418,0.738,PG+ELG,202448,NaN,NaN
2,2,136.639,-0.35203,0.282,0.449,PG+ELG,204140,8.578000e+10,0.2818
3,3,132.368,0.1636,0.128,0.603,PG+ELG,209222,1.425000e+10,0.1277
4,4,132.61,0.11972,0.313,0.608,PG+ELG,209295,8.456000e+10,0.3126
5,5,134.03,0.15244,0.167,0.158,PG+ELG,209584,NaN,NaN
6,6,140.142,0.97341,0.586,0.336,PG+ELG,279956,NaN,NaN
7,7,135.329,1.22984,0.487,0.247,PG+ELG,301818,1.215000e+11,0.4862
8,8,138.941,1.33144,0.593,0.404,PG+ELG,302719,1.474000e+11,0.5933
9,9,130.737,1.55957,0.416,0.35,PG+ELG,323200,1.216000e+11,0.4158


In [133]:
zoo_with_mass

,GAMA_ID,RA,DEC,LENS_SCORE,STELLAR_MASS,Z
0,491457,211.73487,-1.59471,0.000000,3.614000e+10,0.0292
1,460298,211.51452,-1.65754,0.000000,2.086000e+09,0.0542
2,460345,211.78462,-1.71899,0.000000,7.277000e+10,0.1134
3,484623,211.63878,-1.93563,0.000000,4.556000e+10,0.1023
4,484637,211.79006,-1.89456,0.169778,3.489000e+10,0.1029
5,491454,211.71473,-1.54075,0.000000,8.118000e+10,0.0295
6,491414,211.50587,-1.53749,0.000000,1.819000e+10,0.0702
7,484621,211.63321,-1.96885,0.061127,2.298000e+10,0.0351
8,463119,211.54913,-1.22789,0.000000,1.140000e+10,0.0533
9,463148,211.68703,-1.33978,0.143398,9.270000e+09,0.0535


In [134]:
zoo_lens_with_mass

,GAMA_ID,RA,DEC,LENS_SCORE,STELLAR_MASS,Z
0,484637,211.79006,-1.89456,0.169778,3.489000e+10,0.1029
1,484621,211.63321,-1.96885,0.061127,2.298000e+10,0.0351
2,463148,211.68703,-1.33978,0.143398,9.270000e+09,0.0535
3,543438,211.53734,-1.03689,0.213034,8.980000e+09,0.0701
4,463145,211.63664,-1.23923,0.000755,5.946000e+09,0.1378
5,460374,212.03548,-1.70230,0.024226,4.848000e+10,0.0250
6,484655,211.88866,-1.97102,0.003699,1.321000e+10,0.0564
7,460379,211.94664,-1.71637,0.110855,1.077000e+11,0.1136
8,463176,211.81052,-1.19533,0.045798,5.776000e+09,0.0545
9,463197,212.00002,-1.29196,0.093139,1.316000e+11,0.1164


In [135]:
zoo_lens20_with_mass

,GAMA_ID,RA,DEC,LENS_SCORE,STELLAR_MASS,Z
0,543438,211.53734,-1.03689,0.213034,8.980000e+09,0.0701
1,491933,213.94101,-1.42622,0.236187,3.441000e+10,0.1080
2,463601,213.79297,-1.24519,0.221626,8.025000e+10,0.1481
3,492121,214.80569,-1.40320,0.237688,2.811000e+10,0.1022
4,511867,216.38846,-1.11396,0.352229,5.459000e+10,0.0550
5,511865,216.31907,-1.13127,0.315974,2.066000e+10,0.0845
6,485529,216.24766,-1.86855,0.246178,8.986000e+08,0.0302
7,485866,217.55467,-1.72059,0.202707,4.080000e+09,0.0548
8,485873,217.75015,-1.80042,0.349456,4.908000e+10,0.0539
9,508674,217.85610,-1.47254,0.214371,2.560000e+10,0.0915


In [136]:
mac_with_mass

,GAMA_ID,ID,RA,DEC,SCORE,STELLAR_MASS,Z
0,3899011,2251,131.334,-0.91563,64,3.294000e+11,0.4027
1,376338,2660,132.987,1.50468,30,3.034000e+11,0.3272
2,323152,2967,130.546,1.64325,30,8.277000e+11,0.7216
3,663364,2669,140.353,-0.97414,28,2.688000e+11,0.3221
4,3629152,1933,135.889,-0.97487,26,2.129000e+11,0.4059
5,3575500,1906,129.233,-1.57512,26,3.209000e+11,0.3652
6,197972,2007,137.858,-0.7528,24,3.551000e+11,0.3373
7,3883690,2756,134.575,-1.55142,22,1.362000e+11,0.2481
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3911769,1704,138.661,-1.11639,20,1.740000e+11,0.2794


In [142]:
# Plot of Z to Stellar Mass using Zoo with Lens Score > 0.
plt.plot(np.log10(zoo_lens_with_mass.STELLAR_MASS), zoo_lens_with_mass.Z,'y.',label='Galaxy Zoo')
plt.plot(np.log10(mac_with_mass.STELLAR_MASS), mac_with_mass.Z, 'g.',label='Machine Learning')
plt.plot(np.log10(spec_with_mass.STELLAR_MASS), spec_with_mass.Z, 'b.',label='Spectroscopy')

plt.xlabel("Stellar Mass - log(M*)")
plt.ylabel("Z - Redshift")
plt.title("Stellar Mass and Redshift")
plt.legend(loc='best')

plt.show()

In [141]:
# Same plot as above, using Zoo with Lens Score above 20%
plt.plot(np.log10(zoo_lens20_with_mass.STELLAR_MASS), zoo_lens20_with_mass.Z, 'y.',label='Galaxy Zoo')
plt.plot(np.log10(mac_with_mass.STELLAR_MASS), mac_with_mass.Z, 'g.',label='Machine Learning')
plt.plot(np.log10(spec_with_mass.STELLAR_MASS), spec_with_mass.Z, 'b.',label='GAMA Spectroscopy')

plt.xlabel(r"Stellar Mass - $log(M^*/M_\odot)$")
plt.ylabel("Redshift - Z")
plt.title("Stellar Mass and Redshift")
plt.legend(loc='best')

plt.show()

In [146]:
pd.read_csv('/home/shawn/Documents/Lens_Project_Files/mac_in_gama_withgamaid.csv')

,Unnamed: 0,GAMA_ID,ID,RA,DEC,SCORE
0,0,3899011.0,2251.0,131.33397,-0.91563,64.0
1,1,376338.0,2660.0,132.98734,1.50468,30.0
2,2,323152.0,2967.0,130.54643,1.64325,30.0
3,3,663364.0,2669.0,140.35284,-0.97414,28.0
4,4,3629152.0,1933.0,135.88864,-0.97487,26.0
5,5,3575500.0,1906.0,129.23265,-1.57512,26.0
6,6,197972.0,2007.0,137.85838,-0.75280,24.0
7,7,3883690.0,2756.0,134.57492,-1.55142,22.0
8,8,NaN,NaN,NaN,NaN,NaN
9,9,3911769.0,1704.0,138.66089,-1.11639,20.0


In [ ]:
# 6/6 end of day... Check other three csv files, write new notebook with all those collections for ease of access.